In [ ]:
#!pip install -U stable-ts

In [28]:
import stable_whisper
import os
from datetime import datetime
import json


In [33]:
output_path = os.path.join(os.getcwd(), 'output')

if not os.path.exists(output_path):
    os.mkdir(output_path)

model = stable_whisper.load_model('large')


In [37]:
# modified model should run just like the regular model but accepts additional parameters
result = model.transcribe('whisper_audio/vhf2_edited.mp3', verbose=True, word_timestamps= False)
# srt/vtt
# result.to_srt_vtt(output_path + '/audio.srt')
# ass
# result.to_ass(output_path + '/audio.ass')
# json
result.save_as_json(output_path + '/audio.json')

Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: english
[00:00.000 --> 00:09.000]  German Bay, traffic, German Bay, traffic, motor vessel Marfam, Marfam, Bava, Charlie, November, hotel, I'm ready, over.
[00:10.000 --> 00:14.000]  Yeah, Marfam, German Bay, traffic, good afternoon.
[00:15.000 --> 00:23.000]  Good afternoon, sir, motor vessel Marfam, entering the monitoring area, Bowne 4, Bremen, Germany, over.
[00:24.000 --> 00:30.000]  Yeah, coming from Archangels, how many persons on board and your security level?
[00:31.000 --> 00:36.000]  Eight persons on board and security level number one, over.
[00:37.000 --> 00:39.000]  Yeah, what is the draft today?
[00:40.000 --> 00:43.000]  My maximum draft is 5.95, over.
[00:44.000 --> 00:48.000]  Okay, and your present ETA to wizard pilot station?
[00:49.000 --> 00:55.000]  My present ETA to wizard pilot station, 1-7-0-0, local time, over.
[00:58.000 --> 01:04.000]  You will ru

In [21]:

audio_json = json.load(open(output_path + '/audio.json'))

def print_new_line(*text, new_line = 15):
  count = 0
  # every 13 words, print a new line
  line = ""
  for word in text.split():
    line += word + " "
    count += 1
    if count % new_line == 0:
      print(line.strip())
      line = ""
      # Print the remaining words, if any
      if line.strip():
        print(line.strip())

            
print_new_line(audio_json['text'])

German Bay, traffic, German Bay, traffic, motor vessel Marfam, Marfam, Bava, Charlie, November, hotel, I'm
ready, over. Yeah, Marfam, German Bay, traffic, good afternoon. Good afternoon, sir, motor vessel Marfam,
entering the monitoring area, Bowne 4, Bremen, Germany, over. Yeah, coming from Archangels, how many
persons on board and your security level? Eight persons on board and security level number
one, over. Yeah, what is your draft today? My maximum draft is 5.95, over. Okay,
and your present ETA to where is the pilot station? My present ETA to where
is the pilot station, 1700 local time, over. You will run on slow speed to
be at 1700 at the outer pilot station, yeah? Yes, sir, that is correct. As
per agent, our pilot will be 1700 local time, over. Yeah, okay, so all that
for this time might be okay, so have a good watch for some, contact the
pilot, let's say, two hours before arrival on channel 06. Well understood, sir, I will
contact the pilot who are ready for arrival on channe

In [32]:
len(audio_json["segments"])
for seg in audio_json["segments"]:
  start = datetime.strftime(datetime.fromtimestamp(seg["start"]), '%H:%M:%S')
  end = datetime.strftime(datetime.fromtimestamp(seg["end"]), '%H:%M:%S')
  print(start, "->", end, seg["text"])

00:00:00 -> 00:00:08  German Bay, traffic, German Bay, traffic, motor vessel Marfam, Marfam, Bava, Charlie, November, hotel, I'm ready, over.
00:00:10 -> 00:00:13  Yeah, Marfam, German Bay, traffic, good afternoon.
00:00:15 -> 00:00:29  Good afternoon, sir, motor vessel Marfam, entering the monitoring area, Bowne 4, Bremen, Germany, over. Yeah, coming from Archangels, how many persons on board and your security level?
00:00:31 -> 00:00:35  Eight persons on board and security level number one, over.
00:00:37 -> 00:00:38  Yeah, what is your draft today?
00:00:40 -> 00:00:41  My maximum draft is 5
00:00:41 -> 00:00:43 .95, over.
00:00:44 -> 00:00:55  Okay, and your present ETA to where is the pilot station? My present ETA to where is the pilot station, 1700 local time, over.
00:00:58 -> 00:01:04  You will run on slow speed to be at 1700 at the outer pilot station, yeah?
00:01:05 -> 00:01:06  Yes, sir, that is correct.
00:01:07 -> 00:01:08  As per agent,
00:01:09 -> 00:01:15  our pilot wil

In [30]:
for seg in audio_json["segments"]:
  start = datetime.strftime(datetime.fromtimestamp(seg["start"]), '%H:%M:%S')
  end = datetime.strftime(datetime.fromtimestamp(seg["end"]), '%H:%M:%S')
  print_new_line(f'{start} ---> {end} : {seg["text"]}', new_line = 16)

00:00:00 ---> 00:00:08 : German Bay, traffic, German Bay, traffic, motor vessel Marfam, Marfam, Bava, Charlie,
00:00:15 ---> 00:00:29 : Good afternoon, sir, motor vessel Marfam, entering the monitoring area, Bowne 4,
Bremen, Germany, over. Yeah, coming from Archangels, how many persons on board and your security level?
00:00:44 ---> 00:00:55 : Okay, and your present ETA to where is the pilot station? My
00:00:58 ---> 00:01:04 : You will run on slow speed to be at 1700 at the
00:01:09 ---> 00:01:15 : our pilot will be 1700 local time, over. Yeah, okay, so all
00:01:25 ---> 00:01:30 : Well understood, sir, I will contact the pilot who are ready for
